# Twitter Sentiment Analysis - POC
---

## 4. Cleaning Pipeline

So far the steps have been the same as they'd have been for the main project. Settting up a cleaning pipeline is an involved process and the formal beginning of the POC. For the final project, I will not be sampling the data down and all the processing steps will be encapsulated in a custom module.


## POC Only - Sample Data

In [1]:
import re
import os
import time
import json

import numpy as np
import pandas as pd

import urlextract
from html import unescape
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin

# time notebook
start_notebook = time.time()

# load minimally prepared X, y train subsets
raw_path = os.path.join("..","data","1_raw","sentiment140")
X_train = pd.read_csv(os.path.join(raw_path, "X_train.csv"))
y_train = pd.read_csv(os.path.join(raw_path, "y_train.csv"))

In [2]:
# sample down considerably to X, y sample subsets
from sklearn.model_selection import train_test_split

X, X_rest, y, y_rest = train_test_split(X_train, y_train, test_size=0.9999, random_state=158)

The plan is to forget about the `_rest` datasets and focus on the X, y small subsets, as if they were the entire training data.

In [3]:
print(f'Dataset size: {len(X):0.0f}')
print(f'Target distribution: {sum(y["target"])/len(y):0.3f}')

Dataset size: 119
Target distribution: 0.521


In [4]:
X.head(5)

,ID,username,tweet
848825,1827547913,CarissaCruz,"@CassXavier hahaha. yes, i know. it's good fo..."
147277,2204021385,CarlaCh,@shawnieora Been sad lately. Just found out my...
755568,2257571603,Joulez217,@conjunkie ah see my hotel only booked till sa...
15878,1550708931,himmelgarten,Google thinks the Cafe is a spam blog. They'r...
291177,1977512948,murnisitanggang,i love sunday


We need to create an array out of the tweet column, and that array will not contain the shuffled indices in a randomly sampled training dataset so we capture that in a column:

In [5]:
X.insert(3, 'index', X.index)
X.index = range(len(X))
X.head()

,ID,username,tweet,index
0,1827547913,CarissaCruz,"@CassXavier hahaha. yes, i know. it's good fo...",848825
1,2204021385,CarlaCh,@shawnieora Been sad lately. Just found out my...,147277
2,2257571603,Joulez217,@conjunkie ah see my hotel only booked till sa...,755568
3,1550708931,himmelgarten,Google thinks the Cafe is a spam blog. They'r...,15878
4,1977512948,murnisitanggang,i love sunday,291177


In [6]:
# create an array of Tweets
X_array = np.array(X.iloc[:, 2]).ravel()
X_array[:2]

array(["@CassXavier hahaha. yes, i know.  it's good for him. and us! ;)",
       '@shawnieora Been sad lately. Just found out my sister has stage 1 colon cancer. I already lost a sister '],
      dtype=object)

In [7]:
y[:2]

,target
848825,1
147277,0


### Acknowledgements

The contraction map and `expand_contractions` function were adapted from this [KDnuggests article.](https://www.kdnuggets.com/2018/08/practitioners-guide-processing-understanding-text-2.html)

The **DocumentToWordCounterTransformer** class was inspired by Aurelien Geron's **EmailToWordCounterTransformer** class from his famous [classification notebook](https://github.com/ageron/handson-ml/blob/master/03_classification.ipynb), and the **WordCounterToVectorTransformer** is a straight copy of Geron's same class.

In [8]:
def expand_contractions(text, contractions_map):
    
    pattern = re.compile('({})'.format('|'.join(contractions_map.keys())), 
                        flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_map.get(match)\
                                if contractions_map.get(match)\
                                else contractions_map.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def is_ascii(doc):
    try:
        doc.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
           
class DocumentToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, expand_contractions=True, lower_case=True, 
                 replace_usernames=True, unescape_html=True, 
                 replace_urls=True, replace_numbers=True, 
                 remove_junk=True, remove_punctuation=True, 
                 replace_emojis=True, replace_nonascii=True, 
                 remove_stopwords=True, lemmatization=True):
        self.expand_contractions = expand_contractions
        self.lower_case = lower_case
        self.replace_usernames = replace_usernames
        self.unescape_html = unescape_html
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.remove_junk = remove_junk
        self.remove_punctuation = remove_punctuation
        self.replace_emojis = replace_emojis
        self.replace_nonascii = replace_nonascii
        self.remove_stopwords = remove_stopwords
        self.lemmatization = lemmatization
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for doc in X:
            if self.lower_case:
                doc = doc.lower()
            if self.expand_contractions and contractions_map is not None:
                doc = expand_contractions(doc, contractions_map)
            if self.replace_usernames:
                doc = re.sub(r'^@([^\s]+)',' USERNAME ', doc)
            if self.unescape_html:
                doc = unescape(doc)
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(doc)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    doc = doc.replace(url, ' URL ')
            if self.replace_numbers:
                doc = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' NUMBER ', doc)
            if self.remove_junk:
                pattern = r'\¥|\â|\«|\»|\Ñ|\Ð|\¼|\½|\¾|\!|\?|\¿|\x82\
                            |\x83|\x84|\x85|\x86|\x87|\x88|\x89|\
                            |\x8a|\x8b|\x8c|\x8d|\x8e|\°|\µ|\´|\º|\¹|\³'
                doc = re.sub(pattern,'', doc)
            if self.remove_punctuation:
                doc = re.sub(r'\W+', ' ', doc, flags=re.M)
            if self.replace_emojis:
                doc = re.sub(r'[^\x00-\x7F]+', ' EMOJI ', doc)
            if self.replace_nonascii:
                if is_ascii(doc) == False:
                    doc = ' NONASCII '
            word_counts = Counter(doc.split())
            if self.remove_stopwords:
                # 25 semantically non-selective words from the Reuters-RCV1 dataset
                stop_words = ['a','an','and','are','as','at','be','by','for','from',
                              'has','he','in','is','it','its','of','on','that','the',
                              'to','was','were','will','with']
                for word in stop_words:
                    try:
                        word_counts.pop(word)
                    except KeyError:
                        continue
            if self.expand_contractions:
                    leftovers = ['t','s','d','m','ve','re','ll','']
            if self.lemmatization and lemmatizer is not None:
                lemmatized_word_counts = Counter()
                for word, count in word_counts.items():
                    lemmatized_word = lemmatizer.lemmatize(word)
                    lemmatized_word_counts[lemmatized_word] += count
                word_counts = lemmatized_word_counts      
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [9]:
with open("contractions_map.json") as f:
    contractions_map = json.load(f)

url_extractor = urlextract.URLExtract()
lemmatizer = WordNetLemmatizer()

In [10]:
X_wordcounts = DocumentToWordCounterTransformer().fit_transform(X_array)

In [11]:
for counter in X_wordcounts[2:5]:
    counter = str(counter).split("(")[1]
    print(counter)

{'my': 2, 'did': 2, 'USERNAME': 1, 'ah': 1, 'see': 1, 'hotel': 1, 'only': 1, 'booked': 1, 'till': 1, 'sat': 1, 'sorry': 1, 'i': 1, 'not': 1, 'book': 1, 'room': 1, 'another': 1, 'friend': 1, 'who': 1, 'cannot': 1, 'do': 1, 'full': 1, 'weekend': 1})
{'google': 1, 'think': 1, 'cafe': 1, 'spam': 1, 'blog': 1, 'they': 1, 'recognised': 1, 'irrelevant': 1, 'repetitive': 1, 'or': 1, 'nonsensical': 1, 'text': 1, 'told': 1, 'me': 1})
{'i': 1, 'love': 1, 'sunday': 1})


In [12]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [13]:
vocab_transformer = WordCounterToVectorTransformer()
X_vectors = vocab_transformer.fit_transform(X_wordcounts)
X_vectors

<119x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 1208 stored elements in Compressed Sparse Row format>

In [14]:
len(vocab_transformer.vocabulary_)

645

In [15]:
# from original data
X.loc[20:20, ]

,ID,username,tweet,index
20,1562359688,Mizzgp10,@PALMTREEENT lol I'm okay lol but I'm m@dd cuz...,14605


In [16]:
print(X_vectors.toarray()[20][:10])

[0 3 1 0 1 2 1 2 0 0]


In [17]:
for k,v in vocab_transformer.vocabulary_.items():
    if v < 11:
        print(v, k)

1 i
2 USERNAME
3 my
4 NUMBER
5 you
6 not
7 am
8 good
9 day
10 have


In [18]:
print(X_array[20:21])

["@PALMTREEENT lol I'm okay lol but I'm m@dd cuz I don't get 2 meet yall nxt week when yall come down here! "]


In [19]:
print(X_wordcounts[20])

Counter({'i': 3, 'lol': 2, 'am': 2, 'you': 2, 'all': 2, 'USERNAME': 1, 'okay': 1, 'but': 1, 'm': 1, 'dd': 1, 'cuz': 1, 'do': 1, 'not': 1, 'get': 1, 'NUMBER': 1, 'meet': 1, 'nxt': 1, 'week': 1, 'when': 1, 'come': 1, 'down': 1, 'here': 1})


In [20]:
# examine another Tweet
print(X_array[3:4])

["Google thinks the Cafe is a spam blog.  They're recognised by &quot;irrelevant, repetitive, or nonsensical text&quot;.  That's told me "]


In [21]:
X_wordcounts[3]

Counter({'google': 1,
         'think': 1,
         'cafe': 1,
         'spam': 1,
         'blog': 1,
         'they': 1,
         'recognised': 1,
         'irrelevant': 1,
         'repetitive': 1,
         'or': 1,
         'nonsensical': 1,
         'text': 1,
         'told': 1,
         'me': 1})

In [22]:
# found most of the words in this stretch
print(X_vectors.toarray()[3][190:200])

[1 1 1 1 1 1 1 1 1 0]


In [23]:
for k,v in vocab_transformer.vocabulary_.items():
    if v > 189 and v < 201:
        print(v, k)

190 google
191 cafe
192 spam
193 recognised
194 irrelevant
195 repetitive
196 nonsensical
197 text
198 told
199 walkin
200 zoo


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("document_to_wordcount", DocumentToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_array)

In [25]:
X_train_transformed

<119x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 1208 stored elements in Compressed Sparse Row format>

In [26]:
y_array = y.iloc[:,0].ravel()

In [27]:
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_array, cv=5, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[CV]  ................................................................
[CV] .................................... , score=0.708, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.500, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.750, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.583, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.739, total=   0.0s
Mean accuracy: 0.6561594202898551


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [28]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB()
score = cross_val_score(NB_clf, X_train_transformed, y_array, cv=5, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[CV]  ................................................................
[CV] .................................... , score=0.792, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.667, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.667, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.667, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.696, total=   0.0s
Mean accuracy: 0.697463768115942


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


---

In [29]:
# time notebook
mins, secs = divmod(time.time() - start_notebook, 60)
print(f'Total running time: {mins:0.0f} minute(s) and {secs:0.0f} second(s).')

Total running time: 0 minute(s) and 9 second(s).
